In [ ]:
# --- INSTALLS (safe to re-run) ---
import sys, subprocess, pkgutil

def pip_install(pkg):
    if pkg not in {m.name for m in pkgutil.iter_modules()}:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])

for p in ["ccxt", "pandas", "numpy", "ta", "matplotlib", "oandapyV20"]:
    try:
        pip_install(p)
    except Exception as e:
        print(f"Install warning for {p}: {e}")

import ccxt, pandas as pd, numpy as np, time, math, datetime as dt
import matplotlib.pyplot as plt
from ta.trend import EMAIndicator
from ta.volatility import AverageTrueRange

import ccxt


In [ ]:
# ====== USER CONFIG ======
USE_BYBIT   = True     # Crypto via Bybit (paper/live)
USE_OANDA   = False    # Forex via OANDA (set True if you plan to use OANDA)

# Symbols to scan
CRYPTO_SYMBOLS = [
    "BTC/USDT", "ETH/USDT", "SOL/USDT", "BNB/USDT", "XRP/USDT",
    "ADA/USDT", "DOGE/USDT", "LINK/USDT", "AVAX/USDT", "TON/USDT"
]
FOREX_SYMBOLS  = ["EUR/USD", "GBP/USD"]  # used only if USE_OANDA=True

# Risk settings
RISK_PCT = 0.10   # 10% per trade (as requested). For real capital, consider 0.5–2%.
MIN_RR   = 2.0    # min risk:reward

# Timeframes
TF_TREND   = "1d"   # Daily trend filter
TF_SETUP   = "4h"   # Look for pullback/setup
TF_ENTRY   = "1h"   # Entry confirmation

# Backoff to avoid rate limits
SLEEP_SEC = 0.75

# ====== BYBIT API (Testnet recommended) ======
BYBIT_API_KEY    = "YOUR_BYBIT_API_KEY"
BYBIT_API_SECRET = "YOUR_BYBIT_API_SECRET"
BYBIT_TESTNET    = True  # True=testnet, False=live

# ---------- BYBIT (ccxt) ----------
ex_bybit = None
if USE_BYBIT:
    ex_bybit = ccxt.bybit({
        "apiKey": "Enter your API KEY",       ## You will need to create your own API with any broker and Enter the KEY here
        "secret": "Enter your Secret Code",    ## Enter your Secret Code of API
        "enableRateLimit": True,
        "options": {
            "defaultType": "linear",
            "adjustForTimeDifference": True,   # let ccxt compensate
            "recvWindow": 30000
        }
    })
    if BYBIT_TESTNET:
        ex_bybit.set_sandbox_mode(True)
    ex_bybit.load_markets()
    print("Bybit ready (testnet:", BYBIT_TESTNET, ")")


# ---------- OANDA (optional; forex) ----------
if USE_OANDA:
    import oandapyV20
    from oandapyV20 import API
    from oandapyV20.endpoints.accounts import AccountDetails
    from oandapyV20.endpoints.instruments import InstrumentsCandles
    from oandapyV20.endpoints.orders import OrderCreate
    oanda = API(access_token=OANDA_ACCESS_TOKEN, environment=OANDA_ENV)
    print("OANDA ready:", OANDA_ENV)

def fetch_ohlcv_ccxt(ex, symbol, timeframe="1h", limit=500):
    o = ex.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(o, columns=["ts","open","high","low","close","volume"])
    df["ts"] = pd.to_datetime(df["ts"], unit="ms", utc=True).dt.tz_convert("UTC")
    return df.set_index("ts")

def atr(df, period=14):
    ind = AverageTrueRange(df["high"], df["low"], df["close"], window=period, fillna=False)
    return ind.average_true_range()

def ema(df, period):
    return EMAIndicator(close=df["close"], window=period, fillna=False).ema_indicator()


def detect_trend_daily(df_d):
    df = df_d.copy()
    df["ema50"]  = ema(df,50)
    df["ema200"] = ema(df,200)
    last = df.iloc[-1]
    if np.isnan(last["ema200"]) or np.isnan(last["ema50"]):
        return "neutral", "EMAs not ready"
    if last["close"]>last["ema200"] and last["ema50"]>last["ema200"]:
        return "up", "D1 above EMA200 & EMA50>EMA200"
    if last["close"]<last["ema200"] and last["ema50"]<last["ema200"]:
        return "down", "D1 below EMA200 & EMA50<EMA200"
    return "neutral", "Mixed around EMA200"

def recent_swing(df, lookback=20, mode="long"):
    if mode=="long":
        i = df["low"].tail(lookback).idxmin()
        return i, df.loc[i,"low"]
    else:
        i = df["high"].tail(lookback).idxmax()
        return i, df.loc[i,"high"]

def engulfing_signal(df, mode="long"):
    a, b = df.iloc[-2], df.iloc[-1]
    if mode=="long":
        return (b["close"]>b["open"]) and (a["close"]<a["open"]) and (b["close"]>=a["open"]) and (b["open"]<=a["close"])
    else:
        return (b["close"]<b["open"]) and (a["close"]>a["open"]) and (b["close"]<=a["open"]) and (b["open"]>=a["close"])

def breakout_signal(df, mode="long", lookback=20):
    highs = df["high"].tail(lookback+1)[:-1]
    lows  = df["low"].tail(lookback+1)[:-1]
    last = df.iloc[-1]
    if mode=="long":
        return last["close"] > highs.max()
    else:
        return last["close"] < lows.min()

# ---- Tunables (slightly relaxed) ----
NEAR_EMA50_TOLERANCE_UP   = 0.992   # was 0.985 (easier to qualify longs)
NEAR_EMA50_TOLERANCE_DOWN = 1.008   # was 1.015 (easier to qualify shorts)
ATR_BUFFER_MULT = 0.35              # was 0.5 (tighter stops)
ENTRY_CONFIRMS  = ("engulf", "breakout")  # try either

def find_setup(ex, symbol, debug=True):
    msgs = []
    # Daily
    d = fetch_ohlcv_ccxt(ex, symbol, TF_TREND, limit=400)
    trend, reason = detect_trend_daily(d)
    msgs.append(f"D1 trend: {trend} ({reason})")
    if trend=="neutral":
        if debug: print(symbol, "skip:", msgs[-1])
        return None

    # 4H
    h4 = fetch_ohlcv_ccxt(ex, symbol, TF_SETUP, limit=400)
    h4["ema50"] = ema(h4, 50)
    h4["atr14"] = atr(h4, 14)
    last4 = h4.iloc[-1]
    if trend=="up":
        near = last4["close"] >= last4["ema50"]*NEAR_EMA50_TOLERANCE_UP
        if not near:
            if debug: print(symbol, "skip: 4H not near EMA50 for long")
            return None
        mode="long"
    else:
        near = last4["close"] <= last4["ema50"]*NEAR_EMA50_TOLERANCE_DOWN
        if not near:
            if debug: print(symbol, "skip: 4H not near EMA50 for short")
            return None
        mode="short"

    time.sleep(SLEEP_SEC)

    # 1H confirmation
    h1 = fetch_ohlcv_ccxt(ex, symbol, TF_ENTRY, limit=300)
    got_signal = False
    reasons = []
    if "engulf" in ENTRY_CONFIRMS and engulfing_signal(h1, mode=mode):
        got_signal = True
        reasons.append("engulfing")
    if "breakout" in ENTRY_CONFIRMS and breakout_signal(h1, mode=mode, lookback=20):
        got_signal = True
        reasons.append("breakout")

    if not got_signal:
        if debug: print(symbol, "skip: no 1H confirm (engulf/breakout)")
        return None

    # SL/TP
    swing_idx, swing_px = recent_swing(h1, lookback=30, mode=("long" if mode=="long" else "short"))
    atr_val = float(atr(h1,14).iloc[-1])
    atr_buf = atr_val * ATR_BUFFER_MULT
    entry = float(h1.iloc[-1]["close"])

    if mode=="long":
        sl = float(min(swing_px, entry - 1e-6)) - atr_buf
        risk_per_unit = max(1e-6, entry - sl)
        tp = entry + MIN_RR * risk_per_unit
    else:
        sl = float(max(swing_px, entry + 1e-6)) + atr_buf
        risk_per_unit = max(1e-6, sl - entry)
        tp = entry - MIN_RR * risk_per_unit

    plan = {
        "symbol": symbol,
        "side": "buy" if mode=="long" else "sell",
        "entry": entry,
        "sl": round(sl, 6),
        "tp": round(tp, 6),
        "risk_per_unit": risk_per_unit,
        "trend": trend,
        "reasons": reasons,
    }
    if debug:
        print(f"{symbol} ✅ setup: {plan['side']} @ {plan['entry']:.2f} SL {plan['sl']:.2f} TP {plan['tp']:.2f} | reasons={reasons}")
    return plan



def account_equity_bybit(ex):
    bal = ex.fetch_balance()
    # Using USDT linear perps → use total USDT
    eq = bal.get("total",{}).get("USDT") or bal.get("USDT",{}).get("total")
    if eq is None:
        # fallback: sum wallet balances in USDT
        eq = bal.get("USDT",{}).get("free", 0.0)
    return float(eq or 0.0)

def contract_info_bybit(ex, symbol):
    m = ex.market(symbol)
    step = m.get("limits",{}).get("amount",{}).get("min", 0.001)
    precision = m.get("precision",{}).get("amount", 3)
    return step, precision

def round_qty(qty, step, precision):
    if step and step>0:
        qty = math.floor(qty/step)*step
    return float(round(qty, precision))

def size_position_bybit(ex, plan, risk_pct=RISK_PCT):
    equity = account_equity_bybit(ex)
    risk_amt = equity * risk_pct
    step, precision = contract_info_bybit(ex, plan["symbol"])
    qty = risk_amt / plan["risk_per_unit"]  # USDT per $ move on linear is ≈ qty
    qty = round_qty(qty, step, precision)
    return max(qty, step), risk_amt, equity

def place_order_bybit(ex, plan, qty):
    side = plan["side"]
    symbol = plan["symbol"]
    # Market entry with TP/SL attached if supported
    params = {"reduceOnly": False, "timeInForce": "IOC",
              "takeProfit": plan["tp"], "stopLoss": plan["sl"]}
    order = ex.create_order(symbol=symbol, type="market", side=side, amount=qty, params=params)
    return order


def run_scan_and_trade_bybit(symbols, live=False):
    results = []
    for s in symbols:
        try:
            plan = find_setup(ex_bybit, s)
            if not plan: 
                print(f"{s}: no setup")
                continue
            qty, risk_amt, equity = size_position_bybit(ex_bybit, plan, RISK_PCT)
            plan.update({"qty": qty, "risk_amt": round(risk_amt,2), "equity": round(equity,2)})
            print(f"\n{plan['symbol']} | {plan['side'].upper()} | entry≈{plan['entry']:.2f} "
                  f"SL {plan['sl']:.2f} TP {plan['tp']:.2f} | qty {qty} | risk ${risk_amt:.2f} "
                  f"| RR≥{MIN_RR}")
            if live:
                od = place_order_bybit(ex_bybit, plan, qty)
                plan["order_id"] = od.get("id")
                print("→ ORDER SENT:", plan["order_id"])
            results.append(plan)
            time.sleep(SLEEP_SEC)
        except Exception as e:
            print(f"{s}: error {e}")
    return pd.DataFrame(results)

# ---- DRY RUN (no orders) ----
df_plans = run_scan_and_trade_bybit(CRYPTO_SYMBOLS, live=False)
df_plans


if USE_OANDA:
    import json
    def fetch_ohlcv_oanda(instrument="EUR_USD", granularity="H1", count=500):
        params = {"granularity": granularity, "count": count, "price": "M"}
        r = InstrumentsCandles(instrument=instrument.replace("/","_"), params=params)
        oanda.request(r)
        cs = r.response.get("candles", [])
        rows=[]
        for c in cs:
            t = pd.to_datetime(c["time"])
            o = float(c["mid"]["o"]); h=float(c["mid"]["h"]); l=float(c["mid"]["l"]); cl=float(c["mid"]["c"])
            rows.append([t,o,h,l,cl,0.0])
        df = pd.DataFrame(rows, columns=["ts","open","high","low","close","volume"]).set_index("ts")
        return df

    # Example: place market order with SL/TP (units positive=buy, negative=sell)
    def oanda_market_order(account_id, instrument, units, sl, tp):
        data = {
            "order": {
                "instrument": instrument.replace("/","_"),
                "units": str(units),
                "type": "MARKET",
                "positionFill": "DEFAULT",
                "takeProfitOnFill": {"price": str(tp)},
                "stopLossOnFill":  {"price": str(sl)}
            }
        }
        r = OrderCreate(accountID=account_id, data=data)
        return oanda.request(r)


In [1]:
# It is just for taking a testing order on the Demo account


symbol = "ETH/USDT"   # or "ETH/USDT:USDT" if your market dict says so
qty    = 0.001

# 1) get last price
ticker = ex_bybit.fetch_ticker(symbol)
last   = ticker["last"]

# simple demo levels: ~0.6% SL, ~1.2% TP
tp = round(last * 1.012, 2)
sl = round(last * 0.994, 2)

# 2) MARKET BUY
entry = ex_bybit.create_order(
    symbol=symbol,
    type="market",
    side="buy",
    amount=qty,
    params={
        "reduceOnly": False,
        "timeInForce": "IOC",
        "recv_window": 30000,         # help with time drift
        "category": "linear"          # explicit for bybit v5 linear perps
    }
)
entry


# 3) TAKE-PROFIT: reduce-only LIMIT SELL at TP
tp_order = ex_bybit.create_order(
    symbol=symbol,
    type="limit",
    side="sell",
    amount=qty,
    price=tp,
    params={
        "reduceOnly": True,
        "timeInForce": "GTC",
        "recv_window": 30000,
        "category": "linear",
        # Optional: goodTilCancel by default; you can add "orderLinkId" to manage it later
    }
)
tp_order


# 4) STOP-LOSS: reduce-only STOP-MARKET SELL using stopPrice (the "trigger price")
sl_order = ex_bybit.create_order(
    symbol=symbol,
    type="market",            # market on trigger
    side="sell",
    amount=qty,
    price=None,
    params={
        "reduceOnly": True,
        "stopPrice": sl,      # <-- THIS is required (trigger price)
        "triggerBy": "LastPrice",
        "timeInForce": "GTC",
        "recv_window": 30000,
        "category": "linear",
        # "positionIdx": 0,   # one-way mode; uncomment if your account uses position indexes
    }
)
sl_order
